In [9]:
import os
import sys
import re

from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI

import json
import numpy as np
import pandas as pd

from tqdm import tqdm

# Loading functions from the scripts
"""
Mostafa:
I used the new structured output for question generation.
It's a beta version, but it works on my end (10/23/2024).
https://platform.openai.com/docs/guides/structured-outputs/structured-outputs

For answer generation, I had some issues, so I used the standard API."

Please upgrade before running this notebook: pip install --upgrade openai
"""

parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))  # Get parent directory of the notebook 
sys.path.append(parent_dir)  #  to the Python path

from scripts.chunking import process_latex_files
from scripts.embedding import get_embeddings, fixed_knn_retrieval
from scripts.prompts import gen_questions, gen_answer


## 0. Setting API and Models

In [10]:
"""
Mostafa: 
I suggest using 'gpt-4o' for production runs, but it is more expensive.
For embeddings, I recommend 'text-embedding-3-large.' We only need to run it once, but it also costs more.

# https://openai.com/api/pricing/
# https://openai.com/index/new-embedding-models-and-api-updates/
# https://platform.openai.com/docs/guides/embeddings/embedding-models
"""

api_key = os.getenv("OPENAI_API_KEY")  # Replace with your actual API key
client = OpenAI(api_key=api_key)

llm_model = "gpt-4o-mini"
embedding_model = "text-embedding-3-small"    # NOTE: this must be the same for all embeddings. 


# Setting path for root data folder
main_dir = '../data/hughes_latex_Q_then_A_use_context'

if not os.path.exists(main_dir):
    os.makedirs(main_dir)


## 1. Generating Context Embedding Space

In [11]:
"""
Mostafa: 
I used fixed size chunks (512) with a 25% overlap.
Make sure environment_sensitive is set to False for fixed size.

We should embed all chapters to generate the embedding space. For the demo, I only included two chapters.
please update the paths in latex_file_paths.
"""

# add all book chapters paths
latex_file_paths = [ 
    '../data/FEM_Hughes_LaTeX_Textbook/DONE_cao_chapter1.tex',
    '../data/FEM_Hughes_LaTeX_Textbook/DONE_simone_chapter2.tex'
]

tokens_per_chunk = 512                         
token_overlap = int(0.25 * tokens_per_chunk)    # 25% overlap
environment_sensitive = False                   # If False, equations can split between two chunks, but chunk lengths remain fixed.

#------------------------------------------------------------------------
embedding_space_file_name = f'{main_dir}/hughes_latex_embedding_space_tpc{tokens_per_chunk}_o{token_overlap}.json'

space = {}
if not os.path.exists(embedding_space_file_name):
    
    chunks = process_latex_files(latex_file_paths, tokens_per_chunk, token_overlap, environment_sensitive)
    
    # using api
    embedding_space = get_embeddings(client, chunks, model=embedding_model)
    
    # save
    with open(embedding_space_file_name, 'w') as json_file:
        json.dump({'chunks': chunks, 'embedding_space': embedding_space}, json_file)

    print("saved")
else:
    # save
    with open(embedding_space_file_name, 'r') as json_file:
        loaded_data = json.load(json_file)

    chunks = loaded_data['chunks']
    embedding_space = np.array(loaded_data['embedding_space'])
    print("loaded")

chunks = np.array(chunks)
embedding_space = np.array(embedding_space)
print("Space size:", embedding_space.shape)


loaded
Space size: (119, 1536)


## 2. Generating Questions and Their Embeddings

In [12]:
"""
Mostafa: 
For generating questions, we want larger chunks with a bit of overlap.
The following values are just for this demo, so please adjust them as needed.

I only ran Chapter One.
"""

latex_file_path = '../data/FEM_Hughes_LaTeX_Textbook/DONE_cao_chapter1.tex'

chapter = 1
  
max_questions = 6                             # max number of questions per chunk, (small value for this demo)

tokens_per_chunk = 1536                       
token_overlap = int(0.2 * tokens_per_chunk)   # 10% overlap
environment_sensitive = True                  # If True, equations won't be split between chunks, which may result in chunks larger than the specified tokens_per_chunk
                           

#------------------------------------------------------------------------
questions_file_name = f"{main_dir}/hughes_ch{chapter}_Qs_n{max_questions}_tpc{tokens_per_chunk}_o{token_overlap}.json"   

if not os.path.exists(questions_file_name):
    question_chunks = process_latex_files(latex_file_path, tokens_per_chunk, token_overlap, environment_sensitive)

    questions = {}  # main data

    # we should save generation info we used
    questions['info'] = {
        'tokens_per_chunk': tokens_per_chunk,
        'token_overlap': token_overlap,
        'environment_sensitive': environment_sensitive,
        'max_questions': max_questions
    }

    ## step 1: generate questions
    questions['data'] = []
    for i in tqdm(range(len(question_chunks)), desc="Generating Questions"):
        q_for_chunk = gen_questions(client, question_chunks[i], max_questions, model=llm_model)
        questions['data'].append({'chunk': question_chunks[i],'questions': q_for_chunk})
    print('Questions are generated')

    ## step 2: embedding all questions at once
    all_questions = []
    for item in questions['data']:
        for sub_item in item['questions']:
            all_questions.append(sub_item['question'])
    # using api
    embeddings = get_embeddings(client, all_questions, model=embedding_model) 
    # add them to data:
    k = 0
    for item in questions['data']:
        for sub_item in item['questions']:
            sub_item['embedding'] = embeddings[k]
            k +=1
    print('Questions are embedded')
    

    with open(questions_file_name, 'w') as json_file:
        json.dump(questions, json_file, indent=4)
    print('saved', questions_file_name)

else:
    with open(questions_file_name, 'r') as json_file:
        questions = json.load(json_file)

    print('loaded', questions_file_name)


loaded ../data/hughes_latex_Q_then_A_use_context/hughes_ch1_Qs_n6_tpc1536_o307.json


## 3. Context Retrieval and Generating Answers 

In [13]:
"""
Mostafa: 
Since we answer each question separately, this process is slow.
We might want to consider using the batch API for this.
"""

top_k = 3   # number of retrieved closest contexts         

#------------------------------------------------------------------------
questions_answers_file_name = f"{main_dir}/hughes_ch{chapter}_QAs_n{max_questions}_topk{top_k}_tpc{tokens_per_chunk}_o{token_overlap}.json"   

if not os.path.exists(questions_answers_file_name):

    questions_answers = questions.copy()

    # step 1) finding top_k context from the book embedding and adding them to each question
    for item in questions_answers['data']:
        for sub_item in item['questions']:
            ind = fixed_knn_retrieval(sub_item['embedding'], embedding_space, top_k)
            context = ''
            for i, chunk in enumerate(chunks[ind]):
                context += f'\n\n Additional context {i}: {chunk}' 
            sub_item['context'] = context
    print('top_k context added')

    # step 2) generating answers (slow)  (should we try batch API?)
    for item in tqdm(questions_answers['data'], desc="Answering Questions"):
        question_chunk = item['chunk']
        for sub_item in item['questions']:
            question = sub_item['question']
            context = question_chunk + sub_item['context']
            sub_item['answer'] = gen_answer(client, question, context)
    print('Questions are answered')
    
    with open(questions_answers_file_name, 'w') as json_file:
        json.dump(questions_answers, json_file, indent=4)
    print('saved', questions_answers_file_name)

else:
    with open(questions_answers_file_name, 'r') as json_file:
        questions_answers = json.load(json_file)

    print('loaded', questions_answers_file_name)


loaded ../data/hughes_latex_Q_then_A_use_context/hughes_ch1_QAs_n6_topk3_tpc1536_o307.json


## Save a CSV file

In [14]:
"""
Mostafa: 
I think it's better to work with JSON/DataFrame in the code, but for reviewing QAs, CSV is easier to work with
"""

csv_file_name = f"{main_dir}/hughes_ch{chapter}_QAs_n{max_questions}.csv"   
# ----------------------------------

data = []

for item in questions_answers['data']:
    question_chunk = item['chunk']
    for sub_item in item['questions']:
        new_item = {}
        new_item['question_chunk'] = question_chunk
        for k,v in sub_item.items():
            if k == 'embedding':
                continue
            new_item[k] = v
        data.append(new_item)

# data[0]
df = pd.DataFrame(data)[['question_chunk','context','coverage','question','answer']]
df.to_csv(csv_file_name)
df

,question_chunk,context,coverage,question,answer
0,\maketitle \section*{1.1 INTRODUCTORY REMARKS ...,\n\n Additional context 0: \maketitle \section...,80,What are the main constituents of the finite e...,The main constituents of a finite element meth...
1,\maketitle \section*{1.1 INTRODUCTORY REMARKS ...,\n\n Additional context 0: \begin{enumerate} \...,70,Explain the significance of the variational or...,The variational or weak statement of a boundar...
2,\maketitle \section*{1.1 INTRODUCTORY REMARKS ...,\n\n Additional context 0: possibilities. We s...,60,"Given the differential equation $u_{,xx} + f =...","The notation $u_{,xx}$ represents the second d..."
3,\maketitle \section*{1.1 INTRODUCTORY REMARKS ...,"\n\n Additional context 0: formulation, which ...",75,Define the sets of trial solutions and weighti...,In the context of the weak formulation of a bo...
4,\maketitle \section*{1.1 INTRODUCTORY REMARKS ...,\n\n Additional context 0: possibilities. We s...,65,Derive the weak form of the boundary-value pro...,To derive the weak form of the boundary-value ...
...,...,...,...,...,...
61,"(1.I.1) $k$ times. For example, \[ f(x)=\left\...","\n\n Additional context 0: cubics, the first d...",90,Explain the significance of Sobolev spaces in ...,Sobolev spaces play a crucial role in finite e...
62,"(1.I.1) $k$ times. For example, \[ f(x)=\left\...","\n\n Additional context 0: cubics, the first d...",85,Define the Sobolev space $H^{k}$ and describe ...,"The Sobolev space $H^{k}$, denoted as $H^{k}(\..."
63,"(1.I.1) $k$ times. For example, \[ f(x)=\left\...","\n\n Additional context 0: cubics, the first d...",95,What are the implications of Sobolev's theorem...,Sobolev's theorem has significant implications...
64,"(1.I.1) $k$ times. For example, \[ f(x)=\left\...",\n\n Additional context 0: \] is in $C_{b}^{1}...,75,Discuss the concept of generalized derivatives...,"In the context of finite element functions, ge..."


## Print and review

In [15]:
import textwrap

def print_wrapped(text, wrap_length=160):
    wrapped_text = textwrap.fill(text, wrap_length)
    print(wrapped_text)

In [16]:
i = 9  # try different QAs

print('Q:')
print_wrapped(df.iloc[i,:]['question'])
print('A:')
print_wrapped(df.iloc[i,:]['answer'])
print('\nChunk used for Q generation:')
print_wrapped(df.iloc[i,:]['question_chunk'])
print('\nRetrieved context:')
for item in df.iloc[i,:]['context'].split('Additional context'):
    print_wrapped(item)
    print()

Q:
Derive the variational equation from the strong form of the boundary-value problem using integration by parts. What assumptions must be made about the functions
involved?
A:
To derive the variational equation from the strong form of the boundary-value problem, we start with the strong form (S):  \[ \text{(S)} \quad \left\{
\begin{array}{l} u_{,xx} + f = 0 \quad \text{on } \Omega \\ u(1) = g \\ -u_{,x}(0) = h  \end{array}  \right. \]  1. **Assumptions about Functions**:     - We
assume that the functions involved are smooth, meaning they are continuously differentiable. Specifically, we require that the trial solution $u$ and the
weighting function $w$ are in the Sobolev space $H^1$, which implies that their derivatives are square-integrable.  2. **Starting from the Strong Form**:     -
We can rewrite the strong form of the boundary-value problem as:    \[    u_{,xx} + f = 0 \implies u_{,xx} = -f    \]  3. **Multiplying by a Weighting
Function**:     - Multiply both sides of the equa